<a href="https://colab.research.google.com/github/HansAzharr/ML-Material-Screening/blob/main/01_Initial_data_retrieval_and_cleaning_(removed_metadata_widgets).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mp-api pymatgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 25.6 MB/s eta 0:00:00
 

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import json
from google.colab import drive
from mp_api.client import MPRester # Materials Project API client
import getpass
import warnings
import pickle
import time

## Define project paths

In [ ]:
base_project_path = "/content/drive/MyDrive/Colab Notebooks/Masters Research Project"
data_folder_path = os.path.join(base_project_path, "Data")
# Ensure the Data folder exists
os.makedirs(data_folder_path, exist_ok=True)

In [ ]:
# Prompt for API Key using getpass
MP_API_KEY = getpass.getpass("Enter your Materials Project API Key: ")

Enter your Materials Project API Key: ··········


In [ ]:
# Ensure MP_API_KEY and project_root/data_path are available from the previous cell
if 'MP_API_KEY' not in locals() or 'base_project_path' not in locals():
    print("Error: Required variables (MP_API_KEY, base_project_path) not found. Please run the previous cell first.")
    sys.exit("Exiting: Environment not set up.")

# Section 1

## Data retrieval

In [ ]:
def fetch_summary_data(mpr_client, element, fields_to_fetch, energy_hull_filter=(None, 0.2)):
    """Fetches summary data for a given element with specified filters and fields."""
    try:
        docs = mpr_client.materials.summary.search(
            elements=[element],
            energy_above_hull=energy_hull_filter,
            fields=fields_to_fetch
        )
        return [doc.dict() for doc in docs]
    except Exception as e:
        print(f"\n  Error fetching summary data for '{element}': {e}.")
        return []

In [ ]:
def fetch_electrodes_data(mpr_client, element):
    """Fetches electrodes data for a given element (returns all available fields)."""
    try:
        docs = mpr_client.materials.insertion_electrodes.search(
            elements=[element]
        )
        return [doc.dict() if hasattr(doc, 'dict') else doc for doc in docs]
    except Exception as e:
        print(f"\n  Error fetching electrodes data for '{element}': {e}.")
        return []


In [ ]:
# Define elements for query (full list)
all_battery_elements = ["Li", "Na", "Mg", "Al", "O", "S", "P", "F", "Cl", "Co", "Ni", "Mn", "Fe", "Cu", "Zn", "Si", "Ge", "C"]

# Define fields for the general materials summary endpoint
summary_fields_to_fetch = [
    "material_id", "formula_pretty", "elements", "nelements", "volume", "density",
    "energy_above_hull", "band_gap", "is_metal", "total_magnetization", "theoretical"
]

# Define fields for the insertion electrodes endpoint
desired_electrodes_fields = [
    "material_id", "battery_type", "working_ion", "average_voltage",
    "capacity_grav", "capacity_vol", "energy_grav", "energy_vol",
    "num_steps", "max_voltage_step", "stability_charge", "stability_discharge"
]

In [ ]:
warnings.filterwarnings(
    "ignore",
    message="Using UFloat objects with std_dev==0 may give unexpected results.",
    category=UserWarning,
    module='uncertainties.core'
)

In [ ]:
all_summary_docs = []
all_electrodes_docs = []

print(f"Fetching summary fields: {', '.join(summary_fields_to_fetch)}")
print(f"Fetching ALL electrodes fields (will filter later): {', '.join(desired_electrodes_fields)}")

try:
    with MPRester(MP_API_KEY) as m:
        for i, element in enumerate(all_battery_elements):
            print(f"  Processing '{element}' ({i+1}/{len(all_battery_elements)})...", end='\r')

            # Fetch summary data
            summary_data = fetch_summary_data(m, element, summary_fields_to_fetch)
            all_summary_docs.extend(summary_data)

            # Fetch electrodes data (without specifying fields)
            electrodes_data = fetch_electrodes_data(m, element) # CRITICAL FIX: No fields parameter here
            all_electrodes_docs.extend(electrodes_data)

    print(f"\nFinished raw data retrieval. Total summary entries: {len(all_summary_docs)}, Total electrodes entries: {len(all_electrodes_docs)}")

except Exception as e:
    print(f"\n--- CRITICAL ERROR during Data Retrieval ---")
    print(f"Error details: {e}")
    print("Please check API key, network, and query parameters.")
    sys.exit("Exiting: Data retrieval failed.")

Fetching summary fields: material_id, formula_pretty, elements, nelements, volume, density, energy_above_hull, band_gap, is_metal, total_magnetization, theoretical
Fetching ALL electrodes fields (will filter later): material_id, battery_type, working_ion, average_voltage, capacity_grav, capacity_vol, energy_grav, energy_vol, num_steps, max_voltage_step, stability_charge, stability_discharge


Retrieving SummaryDoc documents:   0%|          | 0/19926 [00:00<?, ?it/s]

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/2453 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/11245 [00:00<?, ?it/s]

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/414 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/4811 [00:00<?, ?it/s]

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/72 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/5730 [00:00<?, ?it/s]

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/244 [00:00<?, ?it/s]


Finished raw data retrieval. Total summary entries: 233087, Total electrodes entries: 16338


## Save and Load point

In [ ]:
summary_data_filepath = os.path.join(data_folder_path, "all_summary_docs.pkl")
electrodes_data_filepath = os.path.join(data_folder_path, "all_electrodes_docs.pkl")

print(f"Saving fetched data to '{summary_data_filepath}' and '{electrodes_data_filepath}'...")
try:
    with open(summary_data_filepath, 'wb') as f:
        pickle.dump(all_summary_docs, f)
    with open(electrodes_data_filepath, 'wb') as f:
        pickle.dump(all_electrodes_docs, f)
    print("Data saved successfully.")
except Exception as e:
    print(f"Error saving data: {e}")

Saving fetched data to '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/all_summary_docs.pkl' and '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/all_electrodes_docs.pkl'...
Data saved successfully.


## load data from here

In [ ]:
all_summary_docs = [] # Initialize as empty lists
all_electrodes_docs = [] # Initialize as empty lists

summary_data_filepath = os.path.join(data_folder_path, "all_summary_docs.pkl")
electrodes_data_filepath = os.path.join(data_folder_path, "all_electrodes_docs.pkl")

if os.path.exists(summary_data_filepath) and os.path.exists(electrodes_data_filepath):
    print(f"Loading data from '{summary_data_filepath}' and '{electrodes_data_filepath}'...")
    try:
        with open(summary_data_filepath, 'rb') as f:
            all_summary_docs = pickle.load(f)
        with open(electrodes_data_filepath, 'rb') as f:
            all_electrodes_docs = pickle.load(f)
        print(f"Data loaded successfully. Total summary entries: {len(all_summary_docs)}, Total electrodes entries: {len(all_electrodes_docs)}")
    except Exception as e:
        print(f"Error loading saved data: {e}. Will proceed without loaded data.")
        all_summary_docs = [] # Reset to empty lists if loading fails
        all_electrodes_docs = []
else:
    print("Saved data files not found. Data lists are empty.")

Loading data from '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/all_summary_docs.pkl' and '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/all_electrodes_docs.pkl'...
Data loaded successfully. Total summary entries: 233087, Total electrodes entries: 16338


## Process, Filter, and Combine Raw Data

In [ ]:
# Process Raw Summary Data

if not all_summary_docs:
    print("WARNING: 'all_summary_docs' is empty. Summary DataFrame will be empty.")
    df_summary = pd.DataFrame()
else:
    df_summary = pd.DataFrame(all_summary_docs)
print(f"Initial summary DataFrame has {len(df_summary)} rows.")

if not df_summary.empty:
    # More explicit cleaning for material_id to ensure uniqueness before dropping duplicates
    df_summary['material_id'] = df_summary['material_id'].apply(lambda x: str(x).strip() if pd.notna(x) else np.nan)
    df_summary.dropna(subset=['material_id'], inplace=True) # Drop rows where material_id became NaN after cleaning

    # Keep all desired summary columns here, before the merge
    summary_cols_to_keep = [
        'material_id', 'energy_above_hull', 'elements', 'density',
        'formula_pretty', 'nelements', 'volume', 'band_gap', 'is_metal',
        'total_magnetization', 'theoretical'
    ]
    df_summary_processed = df_summary[[col for col in summary_cols_to_keep if col in df_summary.columns]].copy()

    # Deduplicate summary data. If a material_id appears multiple times in raw summary (e.g. from multiple element searches), keep one.
    df_summary_unique = df_summary_processed.drop_duplicates(subset=['material_id']).copy()

else:
    df_summary_unique = df_summary.copy() # Remains empty if df_summary was empty


Initial summary DataFrame has 233087 rows.


In [ ]:
# Check summary data post-deduplication

print(f"Summary data after initial processing and deduplication: {len(df_summary_unique)} rows.")
print(f"Number of UNIQUE material_ids in df_summary_unique: {df_summary_unique['material_id'].nunique()} (Total rows: {len(df_summary_unique)})")
# Assert to double check - this will raise an error if not unique
if not df_summary_unique.empty:
    assert len(df_summary_unique) == df_summary_unique['material_id'].nunique(), "df_summary_unique still has duplicate material_ids after drop_duplicates!"


Summary data after initial processing and deduplication: 108699 rows.
Number of UNIQUE material_ids in df_summary_unique: 108699 (Total rows: 108699)


In [ ]:
# Process Raw Electrodes Data

if not all_electrodes_docs:
    print("WARNING: 'all_electrodes_docs' is empty. Electrodes DataFrame will be empty.")
    df_electrodes = pd.DataFrame()
else:
    df_electrodes = pd.DataFrame(all_electrodes_docs)
print(f"Initial electrodes DataFrame has {len(df_electrodes)} rows.")

if not df_electrodes.empty:
    # More concise function to extract a clean material_id for linking
    def get_clean_material_id(row):
        if 'adj_pairs' in row and isinstance(row['adj_pairs'], list) and len(row['adj_pairs']) > 0:
            pair = row['adj_pairs'][0]
            material_id_candidate = pair.get('id_charge') or pair.get('id_discharge')
        elif 'material_id' in row and row['material_id']:
            material_id_candidate = row['material_id']
        elif 'material_ids' in row and isinstance(row['material_ids'], list) and len(row['material_ids']) > 0:
            material_id_candidate = row['material_ids'][0]
        else:
            return None

        if material_id_candidate and isinstance(material_id_candidate, str) and (material_id_candidate.startswith('mp-') or material_id_candidate.startswith('mvc-')):
            return material_id_candidate.strip() # Apply strip here
        return None

    df_electrodes['material_id'] = df_electrodes.apply(get_clean_material_id, axis=1)

    # Select desired columns and drop rows with missing material_id, then drop duplicates
    electrode_columns_to_keep = [
        "material_id", "battery_type", "working_ion", "average_voltage",
        "capacity_grav", "capacity_vol", "energy_grav", "energy_vol",
        "num_steps", "max_voltage_step", "stability_charge", "stability_discharge"
    ]
    # Chain operations for conciseness
    df_electrodes_unique = df_electrodes[electrode_columns_to_keep].dropna(
        subset=['material_id']
    ).drop_duplicates(
        subset=['material_id']
    ).copy()


else:
    df_electrodes_unique = pd.DataFrame()

Initial electrodes DataFrame has 16338 rows.


In [ ]:
# DIAGNOSTIC PRINT: Check electrodes data post-deduplication

print(f"Processed electrodes data (cleaned, selected fields, unique material IDs): {len(df_electrodes_unique)} rows.")
print(f"Number of UNIQUE material_ids in df_electrodes_unique: {df_electrodes_unique['material_id'].nunique()} (Total rows: {len(df_electrodes_unique)})")
if not df_electrodes_unique.empty:
    assert len(df_electrodes_unique) == df_electrodes_unique['material_id'].nunique(), "df_electrodes_unique still has duplicate material_ids after drop_duplicates!"


Processed electrodes data (cleaned, selected fields, unique material IDs): 4213 rows.
Number of UNIQUE material_ids in df_electrodes_unique: 4213 (Total rows: 4213)


In [ ]:
# Merge the Processed Summary Data and Processed Electrodes Data using OUTER merge

# FINAL CHECK AND STANDARDIZATION OF material_id JUST BEFORE MERGE (redundant but safe)
if not df_electrodes_unique.empty:
    df_electrodes_unique['material_id'] = df_electrodes_unique['material_id'].apply(lambda x: str(x).strip())
if not df_summary_unique.empty:
    df_summary_unique['material_id'] = df_summary_unique['material_id'].apply(lambda x: str(x).strip())

# Re-verify uniqueness after final strip, just in case
print(f"Post-Final-Strip Check: df_electrodes_unique has {df_electrodes_unique['material_id'].nunique()} unique material_ids out of {len(df_electrodes_unique)} rows.")
print(f"Post-Final-Strip Check: df_summary_unique has {df_summary_unique['material_id'].nunique()} unique material_ids out of {len(df_summary_unique)} rows.")


Post-Final-Strip Check: df_electrodes_unique has 4213 unique material_ids out of 4213 rows.
Post-Final-Strip Check: df_summary_unique has 108699 unique material_ids out of 108699 rows.


In [ ]:
if not df_electrodes_unique.empty or not df_summary_unique.empty:
    df_combined = pd.merge(
        df_summary_unique, # Use the already unique summary data
        df_electrodes_unique, # Use the already unique electrodes data
        on='material_id',
        how='outer', # Keep all materials from both datasets
        suffixes=('_summary', '_electrode') # Differentiate columns if names overlap
    )
    print(f"Combined data after OUTER merge: {len(df_combined)} rows.")

    df_combined = df_combined.drop_duplicates(subset=['material_id']).copy()
    print(f"Combined data after OUTER merge and final material_id deduplication: {len(df_combined)} rows.")

    if 'battery_type' in df_combined.columns:
        initial_rows_before_completeness_filter = len(df_combined)
        df_final_data_filtered_hosts = df_combined[df_combined['battery_type'].notna()].copy()
        print(f"Filtered for completeness of electrode data (e.g., 'battery_type' not null): {len(df_final_data_filtered_hosts)} rows retained (from {initial_rows_before_completeness_filter} rows).")
        print(f"This should be close to the original count of electrode data ({len(df_electrodes_unique)} rows).")
    else:
        print("WARNING: 'battery_type' column not found in df_combined. Cannot filter for completeness of electrode data.")
        df_final_data_filtered_hosts = df_combined.copy() # If battery_type not found, just use df_combined

else:
    print("Both input DataFrames for merging are empty. Skipping merge.")
    df_final_data_filtered_hosts = pd.DataFrame()

Combined data after OUTER merge: 109203 rows.
Combined data after OUTER merge and final material_id deduplication: 109203 rows.
Filtered for completeness of electrode data (e.g., 'battery_type' not null): 4213 rows retained (from 109203 rows).
This should be close to the original count of electrode data (4213 rows).


In [ ]:
# Inspection of df_final_data_filtered_hosts after merge, deduplication, and completeness filter

print("\n--- Inspection of df_final_data_filtered_hosts after merge, deduplication, and completeness filter ---")
print(f"Shape of df_final_data_filtered_hosts: {df_final_data_filtered_hosts.shape}")
print(f"Columns in df_final_data_filtered_hosts: {df_final_data_filtered_hosts.columns.tolist()}")
print("\nInfo on df_final_data_filtered_hosts (including non-null counts and dtypes):")
df_final_data_filtered_hosts.info(verbose=True, show_counts=True) # show_counts will display non-nulls

if 'elements' in df_final_data_filtered_hosts.columns:
    print(f"\nNumber of nulls in 'elements': {df_final_data_filtered_hosts['elements'].isnull().sum()} out of {len(df_final_data_filtered_hosts)} rows.")
    print("First 10 rows of df_final_data_filtered_hosts (material_id, elements):")
    print(df_final_data_filtered_hosts[['material_id', 'elements']].head(10).to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\n'elements' column does not exist in df_final_data_filtered_hosts.")



--- Inspection of df_final_data_filtered_hosts after merge, deduplication, and completeness filter ---
Shape of df_final_data_filtered_hosts: (4213, 22)
Columns in df_final_data_filtered_hosts: ['material_id', 'energy_above_hull', 'elements', 'density', 'formula_pretty', 'nelements', 'volume', 'band_gap', 'is_metal', 'total_magnetization', 'theoretical', 'battery_type', 'working_ion', 'average_voltage', 'capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'num_steps', 'max_voltage_step', 'stability_charge', 'stability_discharge']

Info on df_final_data_filtered_hosts (including non-null counts and dtypes):
<class 'pandas.core.frame.DataFrame'>
Index: 4213 entries, 34 to 109199
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   material_id          4213 non-null   object 
 1   energy_above_hull    3709 non-null   float64
 2   elements             3709 non-null   object 
 3   density         

In [ ]:
#  Final Cleaning and Type Conversion

# Rename Columns to Match Desired Features (from capacity_grav/vol to theoretical_capacity_grav/vol)
df_final_data_filtered_hosts.rename(columns={
    'capacity_grav': 'theoretical_capacity_grav',
    'capacity_vol': 'theoretical_capacity_vol'
}, inplace=True)
print("Renamed 'capacity_grav' to 'theoretical_capacity_grav' and 'capacity_vol' to 'theoretical_capacity_vol'.")


# Convert Numeric Columns
numeric_cols_for_conversion = [
    'average_voltage', # CHANGED from 'average_voltage_electrode'
    'theoretical_capacity_grav', # This is the NEW name after the rename
    'theoretical_capacity_vol',  # This is the NEW name after the rename
    'energy_grav',               # CHANGED from 'energy_grav_electrode'
    'energy_vol',                # CHANGED from 'energy_vol_electrode'
    'num_steps',                 # CHANGED from 'num_steps_electrode'
    'max_voltage_step',          # CHANGED from 'max_voltage_step_electrode'
    'stability_charge',          # CHANGED from 'stability_charge_electrode'
    'stability_discharge',       # CHANGED from 'stability_discharge_electrode'
    'energy_above_hull',         # CHANGED from 'energy_above_hull_summary'
    'density',                   # CHANGED from 'density_summary'
    'nelements',                 # CHANGED from 'nelements_summary'
    'volume',                    # CHANGED from 'volume_summary'
    'band_gap',                  # CHANGED from 'band_gap_summary'
    'total_magnetization'        # CHANGED from 'total_magnetization_summary'
]
for col in numeric_cols_for_conversion:
    if col in df_final_data_filtered_hosts.columns:
        if not pd.api.types.is_numeric_dtype(df_final_data_filtered_hosts[col]):
            df_final_data_filtered_hosts[col] = pd.to_numeric(df_final_data_filtered_hosts[col], errors='coerce')
            if df_final_data_filtered_hosts[col].isnull().sum() > 0:
                print(f"  Warning: Coercing '{col}' to numeric introduced {df_final_data_filtered_hosts[col].isnull().sum()} NaNs.")


Renamed 'capacity_grav' to 'theoretical_capacity_grav' and 'capacity_vol' to 'theoretical_capacity_vol'.


In [ ]:
# Drop rows with NaN in ALL 22 features
# Identify all 22 target features after renaming for the final dropna subset

# These are the columns that MUST have non-null values for downstream modeling.

critical_cols = [
    'material_id',
    'energy_above_hull',
    'elements',
    'density',
    'formula_pretty',
    'nelements',
    'volume',
    'band_gap',
    'is_metal',
    'total_magnetization',
    'theoretical',
    'battery_type',
    'working_ion',
    'average_voltage',
    'theoretical_capacity_grav',
    'theoretical_capacity_vol',
    'energy_grav',
    'energy_vol',
    'num_steps',
    'max_voltage_step',
    'stability_charge',
    'stability_discharge'
]

In [ ]:
# Ensure only drop NaNs for columns that are actually present in the DataFrame
dropna_subset_present = [col for col in critical_cols if col in df_final_data_filtered_hosts.columns]

initial_rows_before_dropna = len(df_final_data_filtered_hosts)
df_final_data_filtered_hosts.dropna(subset=dropna_subset_present, inplace=True)
rows_dropped = initial_rows_before_dropna - len(df_final_data_filtered_hosts)

print(f"Dropped {rows_dropped} rows due to NaN values in critical columns.")
print(f"DataFrame now has {len(df_final_data_filtered_hosts)} rows.")

Dropped 504 rows due to NaN values in critical columns.
DataFrame now has 3709 rows.


In [ ]:
print(f"Final cleaned and processed DataFrame has {len(df_final_data_filtered_hosts)} rows.")
print(f"Final DataFrame columns: {df_final_data_filtered_hosts.columns.tolist()}")
print(f"Number of columns: {len(df_final_data_filtered_hosts.columns)}")


Final cleaned and processed DataFrame has 3709 rows.
Final DataFrame columns: ['material_id', 'energy_above_hull', 'elements', 'density', 'formula_pretty', 'nelements', 'volume', 'band_gap', 'is_metal', 'total_magnetization', 'theoretical', 'battery_type', 'working_ion', 'average_voltage', 'theoretical_capacity_grav', 'theoretical_capacity_vol', 'energy_grav', 'energy_vol', 'num_steps', 'max_voltage_step', 'stability_charge', 'stability_discharge']
Number of columns: 22


In [ ]:
# Display the first few rows
print("\n--- First 5 rows of the final combined and processed DataFrame: ---")

df_final_data_filtered_hosts.head()




--- First 5 rows of the final combined and processed DataFrame: ---


,material_id,energy_above_hull,elements,density,formula_pretty,nelements,volume,band_gap,is_metal,total_magnetization,...,working_ion,average_voltage,theoretical_capacity_grav,theoretical_capacity_vol,energy_grav,energy_vol,num_steps,max_voltage_step,stability_charge,stability_discharge
48,mp-1002568,0.071550,"[Mg, Mn, O]",4.112037,MgMn2O4,3.0,160.058626,0.6493,False,5.997778,...,Na,-0.275526,121.181299,515.367052,-33.388634,-141.997174,1.0,0.000000,0.071550,0.054910
57,mp-10033,0.000000,"[Nb, S]",4.203264,NbS2,2.0,124.077397,0.0000,True,1.727667,...,Li,2.426251,163.446209,698.540768,396.561540,1694.835288,2.0,0.487941,0.000000,0.000000
81,mp-1003637,0.029379,"[Li, Mn, O]",4.350043,LiMn3O6,3.0,102.208625,0.3371,False,10.000594,...,Li,2.341613,49.408746,213.454309,115.696156,499.827361,1.0,0.000000,0.029379,0.059105
95,mp-1003854,0.151309,"[K, Mn, O]",3.781312,KMnO2,3.0,221.390103,1.3322,False,15.999967,...,Mg,0.011719,101.435161,331.113426,1.188689,3.880221,1.0,0.000000,0.151309,0.158791
103,mp-1004375,0.042391,"[Mg, Mn, O]",4.096519,MgMn4O8,3.0,150.812818,0.2766,False,14.000303,...,Na,-0.950239,231.041546,829.097077,-219.544732,-787.840537,1.0,0.000000,0.042391,0.171669


In [ ]:
df_final_data_filtered_hosts.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3709 entries, 48 to 108969
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   material_id                3709 non-null   object 
 1   energy_above_hull          3709 non-null   float64
 2   elements                   3709 non-null   object 
 3   density                    3709 non-null   float64
 4   formula_pretty             3709 non-null   object 
 5   nelements                  3709 non-null   float64
 6   volume                     3709 non-null   float64
 7   band_gap                   3709 non-null   float64
 8   is_metal                   3709 non-null   object 
 9   total_magnetization        3709 non-null   float64
 10  theoretical                3709 non-null   object 
 11  battery_type               3709 non-null   object 
 12  working_ion                3709 non-null   object 
 13  average_voltage            3709 non-null   float64

In [ ]:
df_final_data_filtered_hosts.head(20)

,material_id,energy_above_hull,elements,density,formula_pretty,nelements,volume,band_gap,is_metal,total_magnetization,...,working_ion,average_voltage,theoretical_capacity_grav,theoretical_capacity_vol,energy_grav,energy_vol,num_steps,max_voltage_step,stability_charge,stability_discharge
48,mp-1002568,0.071550,"[Mg, Mn, O]",4.112037,MgMn2O4,3.0,160.058626,0.6493,False,5.997778e+00,...,Na,-0.275526,121.181299,515.367052,-33.388634,-141.997174,1.0,0.000000,0.071550,0.054910
57,mp-10033,0.000000,"[Nb, S]",4.203264,NbS2,2.0,124.077397,0.0000,True,1.727667e+00,...,Li,2.426251,163.446209,698.540768,396.561540,1694.835288,2.0,0.487941,0.000000,0.000000
81,mp-1003637,0.029379,"[Li, Mn, O]",4.350043,LiMn3O6,3.0,102.208625,0.3371,False,1.000059e+01,...,Li,2.341613,49.408746,213.454309,115.696156,499.827361,1.0,0.000000,0.029379,0.059105
95,mp-1003854,0.151309,"[K, Mn, O]",3.781312,KMnO2,3.0,221.390103,1.3322,False,1.599997e+01,...,Mg,0.011719,101.435161,331.113426,1.188689,3.880221,1.0,0.000000,0.151309,0.158791
103,mp-1004375,0.042391,"[Mg, Mn, O]",4.096519,MgMn4O8,3.0,150.812818,0.2766,False,1.400030e+01,...,Na,-0.950239,231.041546,829.097077,-219.544732,-787.840537,1.0,0.000000,0.042391,0.171669
134,mp-10062,0.004804,"[Ba, Cu, Nb, Nd, O]",6.520796,Ba2NdNb(CuO4)2,5.0,195.290587,0.0000,True,1.346100e-03,...,Mg,-1.879517,67.749405,393.215155,-127.336128,-739.054386,1.0,0.000000,0.004804,0.385907
250,mp-1009555,0.192758,"[Cr, O]",5.153050,CrO2,2.0,27.066847,0.0000,True,1.998703e+00,...,Li,-0.826595,294.729377,1423.429227,-243.621754,-1176.599117,1.0,0.000000,0.192758,1.104618
260,mp-1009813,0.191999,"[Si, Sn]",4.361248,SiSn,2.0,55.892187,0.4292,False,5.500000e-06,...,Mg,-0.175472,878.681269,2730.970972,-154.184142,-479.209505,1.0,0.000000,0.191999,0.523977
300,mp-1013526,0.017861,"[S, V]",3.362890,VS2,2.0,56.820383,0.0000,True,1.003633e+00,...,Na,2.258708,194.127443,614.194925,438.477218,1387.287022,1.0,0.000000,0.017861,0.000000
353,mp-1014230,0.000000,"[Ti, Zn]",6.171052,TiZn,2.0,30.480901,0.0000,True,3.000000e-07,...,Zn,0.095410,439.199344,3045.740840,41.904223,290.595618,1.0,0.000000,0.007985,0.007891


In [ ]:
print(df_final_data_filtered_hosts['working_ion'].unique())

['Na' 'Li' 'Mg' 'Zn' 'K' 'Ca' 'Al' 'Y' 'Cs' 'Rb']


In [ ]:
# Get the value counts for the 'working_ion' column
working_ion_counts = df_final_data_filtered_hosts['working_ion'].value_counts()

# Print the counts
print("\nCounts of unique values in 'working_ion':")
print(working_ion_counts)


Counts of unique values in 'working_ion':
working_ion
Li    2287
Mg     994
Na     164
Ca     109
Zn      44
Al      38
K       32
Y       23
Cs      16
Rb       2
Name: count, dtype: int64


In [ ]:
print(f"Final cleaned and processed DataFrame has {len(df_final_data_filtered_hosts)} rows.")
print(f"Final DataFrame columns: {df_final_data_filtered_hosts.columns.tolist()}")
print(f"Number of columns: {len(df_final_data_filtered_hosts.columns)}")

Final cleaned and processed DataFrame has 3709 rows.
Final DataFrame columns: ['material_id', 'energy_above_hull', 'elements', 'density', 'formula_pretty', 'nelements', 'volume', 'band_gap', 'is_metal', 'total_magnetization', 'theoretical', 'battery_type', 'working_ion', 'average_voltage', 'theoretical_capacity_grav', 'theoretical_capacity_vol', 'energy_grav', 'energy_vol', 'num_steps', 'max_voltage_step', 'stability_charge', 'stability_discharge']
Number of columns: 22


## Save the data for feature engineering

In [ ]:
# Save to CSV

if not df_final_data_filtered_hosts.empty:
    output_filename = os.path.join(data_folder_path, "battery_materials_processed.csv")
    df_final_data_filtered_hosts.to_csv(output_filename, index=False)
    print(f"\nFinal processed data saved to '{output_filename}'")
else:
    print("\nDataFrame is empty, no data to save to CSV.")

global _df_final_battery_materials
_df_final_battery_materials = df_final_data_filtered_hosts



Final processed data saved to '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/battery_materials_processed.csv'


# Section 2: New Section for additional data retrieval

This new section was added to enrich the existing dataset (battery_materials_processed.csv) with additional material properties from the Materials Project API. Based on insights gained from preliminary model analysis (e.g., error patterns in Notebook 03), the following key features were identified as potentially beneficial for improving model performance:

- efermi: Fermi energy, crucial for understanding metallic character.

- nsites: The number of atoms in the primitive cell, providing structural scale information.

- crystal_system: The crystal lattice system (e.g., cubic, hexagonal), offering fundamental structural classification.

Instead of re-running the entire initial data collection, this section efficiently retrieves only these specific features for the existing materials, merges them into the DataFrame, and handles any resulting missing values. The enhanced dataset is then saved as battery_materials_processed_v2.csv, ready for use in subsequent notebooks.

In [ ]:
#  Load existing processed DataFrame from CSV

INPUT_CSV_FILENAME = "battery_materials_processed.csv"
OUTPUT_CSV_FILENAME = "battery_materials_processed_v2.csv"

input_filepath = os.path.join(data_folder_path, INPUT_CSV_FILENAME)

try:
    df_to_enhance = pd.read_csv(input_filepath)
    print(f"Loaded '{INPUT_CSV_FILENAME}' from '{data_folder_path}' (shape: {df_to_enhance.shape}).")
except FileNotFoundError:
    raise FileNotFoundError(f"ERROR: '{input_filepath}' not found. Ensure previous Notebook 01 steps ran and saved this file.")

if 'material_id' not in df_to_enhance.columns:
    raise ValueError("'material_id' column not found. It's required for MP API lookup.")


Loaded 'battery_materials_processed.csv' from '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data' (shape: (3709, 22)).


In [ ]:
# Prompt for API Key using getpass
MP_API_KEY = getpass.getpass("Enter your Materials Project API Key: ")

Enter your Materials Project API Key: ··········


In [ ]:
# Initialize MPRester
try:
    mpr = MPRester(MP_API_KEY)
except Exception as e:
    raise ConnectionError(f"ERROR: Failed to initialize MPRester. Check API key and internet: {e}")


## further data retrieval

In [ ]:
# Define Fields and Retrieve Data in Batches

mp_fields = ["material_id", "efermi", "nsites", "symmetry.crystal_system"]
mp_ids = df_to_enhance['material_id'].unique().tolist()
retrieved_data = []

print(f"Fetching {len(mp_ids)} materials from MP in batches...")
batch_size = 1000
for i in range(0, len(mp_ids), batch_size):
    batch = mp_ids[i : i + batch_size]
    try:
        docs = mpr.materials.summary.search(material_ids=batch, fields=mp_fields)
        for doc in docs:
            d = doc.dict()
            retrieved_data.append({
                'material_id': d.get('material_id'),
                'e_fermi': d.get('efermi'),
                'nsites': d.get('nsites'),
                'crystal_system': d.get('symmetry', {}).get('crystal_system', np.nan)
            })
    except Exception as e:
        print(f"WARNING: Batch error for IDs {batch[0]}...{batch[-1]}: {e}")
    time.sleep(0.1)

mp_features_df = pd.DataFrame(retrieved_data)
print(f"Retrieved data for {len(mp_features_df)} materials.")

Fetching 3709 materials from MP in batches...


Retrieving SummaryDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/709 [00:00<?, ?it/s]

Retrieved data for 3709 materials.


In [ ]:
# Merge New Data with Existing DataFrame

df_enhanced = pd.merge(df_to_enhance, mp_features_df, on='material_id', how='left')
print(f"DataFrame shape after merge: {df_enhanced.shape}")

DataFrame shape after merge: (3709, 25)


In [ ]:
# Handle Missing Values in New Features
print("\nHandling missing values in new MP features:")
print(f"Before imputation:\n{df_enhanced[['e_fermi', 'nsites', 'crystal_system']].isnull().sum()}")



Handling missing values in new MP features:
Before imputation:
e_fermi           2
nsites            0
crystal_system    0
dtype: int64


In [ ]:
# Impute e_fermi (Numerical)
if df_enhanced['e_fermi'].isnull().any():
    df_enhanced['e_fermi'] = df_enhanced['e_fermi'].fillna(df_enhanced['e_fermi'].median())
    print(" - 'e_fermi' imputed with median.")

# Impute nsites (Numerical, Integer)
if df_enhanced['nsites'].isnull().any():
    df_enhanced['nsites'] = df_enhanced['nsites'].fillna(df_enhanced['nsites'].median())
    df_enhanced['nsites'] = df_enhanced['nsites'].astype(int)
    print(" - 'nsites' imputed with median and converted to int.")

# Impute crystal_system (Categorical)
if df_enhanced['crystal_system'].isnull().any():
    df_enhanced['crystal_system'] = df_enhanced['crystal_system'].fillna(df_enhanced['crystal_system'].mode()[0])
    print(" - 'crystal_system' imputed with mode.")

print(f"After imputation:\n{df_enhanced[['e_fermi', 'nsites', 'crystal_system']].isnull().sum()}")

 - 'e_fermi' imputed with median.
After imputation:
e_fermi           0
nsites            0
crystal_system    0
dtype: int64


In [ ]:
df_enhanced.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3709 entries, 0 to 3708
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   material_id                3709 non-null   object 
 1   energy_above_hull          3709 non-null   float64
 2   elements                   3709 non-null   object 
 3   density                    3709 non-null   float64
 4   formula_pretty             3709 non-null   object 
 5   nelements                  3709 non-null   float64
 6   volume                     3709 non-null   float64
 7   band_gap                   3709 non-null   float64
 8   is_metal                   3709 non-null   bool   
 9   total_magnetization        3709 non-null   float64
 10  theoretical                3709 non-null   bool   
 11  battery_type               3709 non-null   object 
 12  working_ion                3709 non-null   object 
 13  average_voltage            3709 non-null   float

## Save updated dataset

In [ ]:
# Save the Enhanced DataFrame

output_filepath = os.path.join(data_folder_path, OUTPUT_CSV_FILENAME)
if not df_enhanced.empty:
    df_enhanced.to_csv(output_filepath, index=False)
    print(f"\nEnhanced DataFrame saved to '{output_filepath}'.")
else:
    print("DataFrame is empty, no data to save.")


Enhanced DataFrame saved to '/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data/battery_materials_processed_v2.csv'.


## Appendix: Data Retrieval and Processing

This appendix details the steps taken to retrieve, process, and clean the battery materials data from the Materials Project API (using `mp-api`) for use in this study.

### Data Sources

Data was primarily retrieved from two key endpoints of the Materials Project API:

1.  **Materials Summary (`mpr.materials.summary`):** This endpoint provides general, fundamental properties of materials.
2.  **Insertion Electrodes (`mpr.materials.insertion_electrodes`):** This endpoint provides properties specifically calculated for materials acting as insertion electrodes in battery applications.

### Data Retrieval Process

The data was retrieved using the `MPRester` client from the `mp-api` library. A list of battery-relevant elements (`all_battery_elements`) was defined to filter the search space to materials likely to be of interest for battery applications.

The following fields were initially targeted from the Materials Summary endpoint:

- `material_id`: Unique identifier for the material.
- `formula_pretty`: The conventional formula string.
- `elements`: List of elements in the material.
- `nelements`: Number of unique elements.
- `volume`: Unit cell volume.
- `density`: Density of the material.
- `energy_above_hull`: Energy above the convex hull, indicating thermodynamic stability.
- `band_gap`: Electronic band gap.
- `is_metal`: Boolean indicating if the material is metallic.
- `total_magnetization`: Total magnetic moment.
- `theoretical`: Boolean indicating if the material is theoretical.

From the Insertion Electrodes endpoint, all available fields were initially fetched, and a subset (`desired_electrodes_fields`) was later selected based on relevance:

- `material_id`: Unique identifier for the material (linked via constituent materials).
- `battery_type`: Type of battery (e.g., insertion).
- `working_ion`: The ion inserted/extracted (e.g., Li, Na, Mg).
- `average_voltage`: Average voltage during insertion/extraction.
- `capacity_grav`: Gravimetric theoretical capacity.
- `capacity_vol`: Volumetric theoretical capacity.
- `energy_grav`: Gravimetric energy density.
- `energy_vol`: Volumetric energy density.
- `num_steps`: Number of voltage steps in the insertion/extraction curve.
- `max_voltage_step`: Maximum voltage step.
- `stability_charge`: Stability during charging.
- `stability_discharge`: Stability during discharging.

Data for each element in `all_battery_elements` was fetched iteratively using the `fetch_summary_data` and `fetch_electrodes_data` helper functions. The retrieved data was stored as lists of dictionaries: `all_summary_docs` and `all_electrodes_docs`.

### Data Persistence

To avoid repeated API calls, the raw retrieved data (`all_summary_docs` and `all_electrodes_docs`) was saved to pickle files (`all_summary_docs.pkl` and `all_electrodes_docs.pkl`) in the designated data folder (`/content/drive/MyDrive/Colab Notebooks/Masters Research Project/Data`). A loading mechanism was also implemented to load the data from these files if they exist, allowing the user to restart the notebook without refetching.

### Data Processing and Cleaning

The raw data lists were converted into pandas DataFrames (`df_summary` and `df_electrodes`).

1.  **Material ID Cleaning and Deduplication:** Material IDs were standardized by stripping whitespace, and duplicate entries within each DataFrame were removed using `drop_duplicates` based on the `material_id`. This resulted in `df_summary_unique` and `df_electrodes_unique`, ensuring each material was represented only once in each source.

2.  **Data Merging:** The unique summary and electrodes DataFrames were merged using an outer join (`how='outer'`) on the `material_id`. This step aimed to combine properties from both sources for materials present in either dataset. The resulting DataFrame was named `df_combined`. A final `drop_duplicates` on `material_id` was applied to `df_combined` as a safeguard.

3.  **Completeness Filtering:** The merged DataFrame was filtered to retain only rows where the `battery_type` column was not null. This step was crucial for ensuring the final dataset contained only materials with calculated battery insertion properties, resulting in `df_final_data_filtered_hosts`.

4.  **Column Renaming and Type Conversion:** Several columns were renamed for clarity, specifically:
    - `capacity_grav` to `theoretical_capacity_grav`
    - `capacity_vol` to `theoretical_capacity_vol`
    Relevant columns were converted to numeric data types using `pd.to_numeric` with `errors='coerce'` to handle potential non-numeric values gracefully.

5.  **Handling Missing Values:** Rows with missing values in a predefined list of 22 critical features (including the newly renamed and numeric columns) were removed using `dropna(subset=...)`. This ensured that the final dataset was complete for the features intended for use in downstream analysis and modeling.

### Additional Feature Retrieval

In a subsequent step (Section 2 of the notebook), additional potentially relevant features were retrieved for the materials already present in the processed dataset (`battery_materials_processed.csv`). These features, identified based on preliminary analysis, include:

- `efermi`: Fermi energy.
- `nsites`: Number of sites in the primitive cell.
- `symmetry.crystal_system`: The crystal system.

These features were fetched in batches using `mpr.materials.summary.search` with the `material_ids` from the existing DataFrame and the specified `fields`. The retrieved data was merged with the existing DataFrame (`df_to_enhance`) using a left merge on `material_id`, resulting in `df_enhanced`. Missing values in these new columns (`e_fermi`, `nsites`, `crystal_system`) were imputed using the median for numeric columns (`e_fermi`, `nsites`) and the mode for the categorical column (`crystal_system`).

### Final Dataset

The final cleaned and processed DataFrame (`df_enhanced`, saved as `battery_materials_processed_v2.csv`) contains {{len(df_enhanced)}} entries and {{len(df_enhanced.columns)}} features. This dataset represents materials from the Materials Project with both fundamental and battery-specific properties, cleaned and prepared for feature engineering and machine learning model development.

An overview of the distribution of `working_ion` in the final dataset is provided below:

{{df_enhanced['working_ion'].value_counts().to_markdown(numalign="left", stralign="left")}}